In [18]:
import pandas as pd
from urllib.parse import quote
from google.colab import files
import numpy as np
import time

In [41]:
from tqdm import tqdm

In [19]:
time.sleep(1)

In [20]:
df = pd.read_csv('/content/1923_dramalist.csv').iloc[:,1:]
df.loc[df['작품명'] == '멀리서 보는 푸른 봄', '작품명'] = '멀리서 보면 푸른 봄'
drama_list = df['작품명'].tolist()
drama_list.extend(['에이틴', '에이틴 2'])

In [43]:
meta1=pd.DataFrame()

for name in tqdm(drama_list):
  time.sleep(0.2)
  title = pd.DataFrame({"작품명":[name]})
  try:

    url = f"https://ko.wikipedia.org/wiki/{name}_(드라마)"
    encoded_url = quote(url, safe=':/_')
    tables = pd.read_html(encoded_url)

    # 수정 )) 이렇게 해야 다음으로 넘어갈 수 있는데 제가 실수했습니다. 아래의 try는 굳이 바꿀 필요 없어서 냅뒀어요.
    for i in tables:
      if i.iloc[2,0] == '장르' or i.iloc[1,0] == '장르':
        table = i
        break

    table = table[0:].dropna().T
    table.columns = table.iloc[0]
    table = table[1:]

    desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

    for col in desired:
      if col not in table.columns:
        table[col] = np.nan

    table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

    table = pd.concat([title,table],axis=1)

    # 이전 이후 드라마
    df3 = pd.DataFrame()
    for i in tables:
      try:
        if i.columns[0][1] == '이전 작품':
          df3 = i
          df3.columns = ['이전', '현재', '다음']
          break
      except Exception:
        pass

    table = pd.concat([table, df3], axis=1)

  except Exception:

    try:
      url = f"https://ko.wikipedia.org/wiki/{name}"
      encoded_url = quote(url, safe=':/_')
      tables = pd.read_html(encoded_url)
      table = tables[0]

      table = table[0:].dropna().T
      table.columns = table.iloc[0]
      table = table[1:]

      desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

      for col in desired:
        if col not in table.columns:
          table[col] = np.nan

      table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

      table = pd.concat([title,table],axis=1)

      # 이전 이후 드라마
      df3 = pd.DataFrame()
      for i in tables:
        try:
          if i.columns[0][1] == '이전 작품':
            df3 = i
            df3.columns = ['이전', '현재', '다음']
            break
        except Exception:
          pass

      table = pd.concat([table, df3], axis=1)

    except Exception as e:
      table = title
      print(name, e)

  meta1=pd.concat([meta1,table],ignore_index=True) # concat 함수 내부 meta -> meta1


  0%|          | 2/422 [00:01<04:09,  1.69it/s]

          작품명
0  너를 싫어하는 방법 HTTP Error 404: Not Found


  1%|▏         | 6/422 [00:03<04:00,  1.73it/s]

           작품명
0  각색은 이미 시작됐다 HTTP Error 404: Not Found


  2%|▏         | 10/422 [00:05<04:02,  1.70it/s]

...

 22%|██▏       | 94/422 [00:40<01:58,  2.76it/s]

              작품명
0  에이틴 / 에이틴 시즌 2 HTTP Error 404: Not Found


 50%|█████     | 213/422 [01:26<01:32,  2.25it/s]

        작품명
0  아이를 찾습니다 HTTP Error 404: Not Found


100%|██████████| 422/422 [02:53<00:00,  2.43it/s]


In [45]:
errors = meta1[meta1['장르'].isna()]["작품명"].unique().tolist()
print(errors)
## 코드를 약간 수정한 결과 에러리스트가 대폭 감소하였습니다.

['신비아파트 외전: 기억, 하리 2', nan, '너를 싫어하는 방법', '각색은 이미 시작됐다', '반야', '개같다 거지같다 아름답다', '빅이슈', '인서울 - 내가 독립하는 유일한 방법', '에이틴 / 에이틴 시즌 2', '엑스가리온', '터치', '나의 위험한 아내', '트웬티트웬티', '하이에나', '바람과 구름과 비', '행복의 진수', '외출', '악의 꽃', '괴물', '아이를 찾습니다', '언더커버', '지리산', '내일', '아직 최선을 다하지 않았을 뿐', '이브', '괴이', '블라인드', '장미맨션', '치얼 업', '커튼 콜', '결혼백서', '아일랜드']


In [73]:
meta1 = meta1.drop('현재', axis = 1)
meta1 = meta1.drop([37,47,48,49,372]) # 외국작품인 리갈하이, 연예인매니저로 살아남기, 파고 삭제

In [74]:
# 검색 결과 어차피 안 나오는 것들은 배제하였습니다.
error_list = ['빅이슈 (2019년 드라마)', '터치 (2020년 드라마)', '나의 위험한 아내 (대한민국의 드라마)', '트웬티 트웬티', '하이에나 (2020년 드라마)', '바람과 구름과 비 (2020년 드라마)',
              '외출 (2020년 드라마)', '악의 꽃 (2020년 드라마)', '괴물 (2021년 드라마)', '언더커버 (2021년 드라마)', '지리산 (2021년 드라마)', '내일 (2022년 드라마)',
              '이브 (2022년 드라마)', '블라인드 (2022년 드라마)', '커튼콜 (드라마)', '치얼업 (드라마)', '아일랜드 (2022년 드라마)', '리갈 하이 (2019년 드라마)', '연예인 매니저로 살아남기 (대한민국의 드라마)']


In [75]:
# 사실 에러리스트를 만든 이상 try구문은 필요 없습니다. 에러리스트는 손수 구한 정확한 단어들이어서, 오류가 생기지 않거든요.
# 그러나 이하에서는 코드를 더 안 건드리겠습니다. 맨 마지막 concat 안을 meta2로만 바꿨습니다.
meta2=pd.DataFrame()

for name in error_list:
  title = pd.DataFrame({"작품명":[name]})
  try:

    url = f"https://ko.wikipedia.org/wiki/{name}"
    encoded_url = quote(url, safe=':/_')
    tables = pd.read_html(encoded_url)
    table = tables[0]

    table = table[0:].dropna().T
    table.columns = table.iloc[0]
    table = table[1:]

    desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

    for col in desired:
      if col not in table.columns:
        table[col] = np.nan

    table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

    table = pd.concat([title,table],axis=1)

        # 이전 이후 드라마
    for i in tables:
      try:
        if i.columns[0][1] == '이전 작품':
          df3 = i
          break
      except Exception:
        pass
    df3.columns = ['이전', '현재', '다음']
    table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

  except Exception:

    try:
      url = f"https://ko.wikipedia.org/wiki/{name}_(드라마)"
      encoded_url = quote(url, safe=':/_')
      tables = pd.read_html(encoded_url)
      table=tables[0]

      table = table[0:].dropna().T
      table.columns = table.iloc[0]
      table = table[1:]

      desired = ['작품명', '장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']

      for col in desired:
        if col not in table.columns:
          table[col] = np.nan

      table = table[['장르', '방송 채널', '방송 기간', '방송 시간', '방송 분량', '방송 횟수', '기획', '책임프로듀서', '프로듀서', '제작사', '연출', '각본', '출연자']].reset_index(drop = True)

      table = pd.concat([title,table],axis=1)

            # 이전 이후 드라마
      for i in tables:
        try:
          if i.columns[0][1] == '이전 작품':
            df3 = i
            break
        except Exception:
          pass
      df3.columns = ['이전', '현재', '다음']
      table = pd.concat([table, df3.iloc[:, [0,2]]], axis=1)

    except Exception:
      table = title

  meta2=pd.concat([meta2,table],ignore_index=True)


In [84]:
meta = pd.concat([meta1, meta2], ignore_index = True)
meta = meta[~meta['작품명'].isna()].reset_index(drop = True)

In [85]:
meta.to_csv('메타데이터.csv', index=False, encoding='utf-8-sig')
# files.download('미니시리즈_메타데이터.csv')